# ABOUT:

In [32]:
from recommenders.models.sar import SAR
from recommenders.datasets.python_splitters import python_stratified_split
from recommenders.utils.constants import SEED as DEFAULT_SEED
from recommenders.evaluation.python_evaluation import precision_at_k
from recommenders.models.deeprec.DataModel.ImplicitCF import ImplicitCF
from recommenders.models.deeprec.deeprec_utils import prepare_hparams
from recommenders.models.deeprec.models.graphrec.lightgcn import LightGCN
from recommenders.utils.timer import Timer

import pandas as pd
import os
import gzip
import json
import pandas as pd
import seaborn as sb
from scipy.sparse import save_npz, load_npz
from numpy import save, load 
import json
import numpy as np
import random
import pickle
from math import ceil

import sys
sys.path.append(r'C:\Users\tanch\Desktop\Bot.World\Bot.World\src\main\api\fastapi')
sys.path.append(r'C:\Users\tanch\Desktop\Bot.World\Bot.World\src\main\api')
from model import load_sar_model
from bot_world_config import *

### bot_world_config

In [2]:
CONFIG = {}
CONFIG.update(load_database_config())
CONFIG.update(load_model_config())
CONFIG.update(load_session_config())
CONFIG.update(load_seller_config())
CONFIG.update(load_sar_config())
CONFIG.update(load_telegram_bot_config())

In [3]:
CONFIG

{'DATABASE_USER': 'ching',
 'DATABASE_PASSWORD': '978775!Rtj01',
 'DATABASE_HOST': 'aws-bot-world-mysql.c6ecc5e9b7no.ap-southeast-1.rds.amazonaws.com',
 'DATABASE_NAME': 'Arietes_Care',
 'PRODUCT_INFO_TABLE_NAME': 'product_info',
 'USE_RDS': False,
 'DATABASE_PATH_TANCH': 'C:\\Users\\tanch\\Desktop\\Bot.World\\Bot.World\\src\\main\\database\\arietes\\arietes.db',
 'DATABASE_PATH_DOCKER': '/database/arietes/arietes.db',
 'MODEL_TYPE': 'sar',
 'TOP_K': 10,
 'MODEL_SUPPORTED_TYPES': ['sar'],
 'SESSION_RECOMMENDATION_SIZE': 5,
 'SELLER_ID': '120171733',
 'SHOP_NAME': 'Arietes Care',
 'SHOP_URL': 'https://shopee.sg/arietes.acc',
 'COL_USER': 'user_id',
 'COL_ITEM': 'item_id',
 'COL_RATING': 'rating',
 'SIMILARITY_TYPE': 'jaccard',
 'MODEL_DIR': '/api/models/arietes-sar',
 'BOT_TOKEN': '6152761301:AAE-VFcRJn7r40lbNOEC5QX5rZCY4DjGULA'}

### config

In [4]:
KEYWORD_SHOPEE_RAW_TRANSACTION_FILE = "Order.all"
NON_KEYWORD_SHOPEE_RAW_TRANSACTION_FILE = "$"
DESIRED_COLS_TRANSACTION_FILE = ["Order ID", "Original Price", "Username (Buyer)", "Product Name", "product_id"]
TRANSACTION_ID_COL = ["Order ID"]
# USER_ID_COL = "user_id"
# ITEM_ID_COL = "item_id"
# RATING_COL = "rating"
TRANSACTION_TO_INTERACTION_MAPPING = {
    CONFIG['COL_USER']:"Username (Buyer)",
    CONFIG['COL_ITEM']:"product_id",
}

PATH_TO_RAW_MEDIA_FILE = r"C:\Users\tanch\Documents\NTU\NTU Year 4\FYP - GNN\data\shopee-20221117T041452Z-001\shopee\mass_update_media_info_120171733_20230224232836.xlsx"
PATH_TO_RAW_TRANSACTIONS_FOLDER = r"C:\Users\tanch\Documents\NTU\NTU Year 4\FYP - GNN\data\shopee-20221117T041452Z-001\shopee"

### model training config

In [5]:
MIN_UNIQUE_PURCHASES = 2
SPLIT_RATIO = 0.5
SEED = 0 
TOP_K = 10
BATCH_SIZE = 1024
MODEL_DIR = r"C:\Users\tanch\Desktop\Bot.World\Bot.World\src\main\api\models\arietes-sar"
MODEL_TRAINING_DATA_DIR = r"C:\Users\tanch\Desktop\Bot.World\Bot.World\src\dev\data\recommendation model training\arietes"

## Load data and model

In [6]:
train = pd.read_pickle(os.path.join(MODEL_TRAINING_DATA_DIR, "train.pkl"))
test = pd.read_pickle(os.path.join(MODEL_TRAINING_DATA_DIR, "test.pkl"))

In [7]:
model = load_sar_model(CONFIG)

## predict by popularity

In [8]:
model.get_popularity_based_topk(top_k=CONFIG["SESSION_RECOMMENDATION_SIZE"])

,item_id,prediction
0,3333330067,785
1,20901098335,691
2,13671455179,556
3,15180905833,387
4,5834933106,317


In [10]:
train.item_id.value_counts()

3333330067     785
20901098335    691
13671455179    556
15180905833    387
5834933106     317
              ... 
4772763551       1
21828301735      1
22306741687      1
18106970802      1
21433429878      1
Name: item_id, Length: 249, dtype: int64

## predict by user

In [12]:
test_users = ["bignphat"]

In [13]:
model.recommend_k_items(test = pd.DataFrame({CONFIG["COL_USER"]:test_users}), 
                                              top_k=CONFIG["SESSION_RECOMMENDATION_SIZE"], 
                                              remove_seen=True)

,user_id,item_id,prediction
0,bignphat,16739500035,0.004016
1,bignphat,18107316966,0.000769
2,bignphat,18907250016,0.000600
3,bignphat,7936874543,0.000450
4,bignphat,12792701275,0.000309


## predict by items

In [18]:
test_items = ["13950585519"]

In [20]:
model.get_item_based_topk(items = pd.DataFrame({CONFIG["COL_ITEM"]:test_items}), 
                                                top_k=CONFIG["SESSION_RECOMMENDATION_SIZE"])

,user_id,item_id,prediction
0,0,18414274019,0.0
1,0,21433429878,0.0


## hyrbid - popularity and item

In [27]:
test_items = ["13950585519"]

In [34]:
model.get_popularity_based_topk(top_k=ceil(CONFIG["SESSION_RECOMMENDATION_SIZE"]/2))


,item_id,prediction
0,3333330067,785
1,20901098335,691
2,13671455179,556


In [ ]:
model.get_item_based_topk(items = pd.DataFrame({CONFIG["COL_ITEM"]:test_items}), 
                                                top_k=CONFIG["SESSION_RECOMMENDATION_SIZE"]//2)

,user_id,item_id,prediction
0,0,18414274019,0.0
1,0,21433429878,0.0


## hybrid - popularity and user

In [ ]:
test_users = ["bignphat"]

In [ ]:
model.get_popularity_based_topk(top_k=ceil(CONFIG["SESSION_RECOMMENDATION_SIZE"]/2))

,item_id,prediction
0,3333330067,785
1,20901098335,691
2,13671455179,556


In [ ]:
model.recommend_k_items(test = pd.DataFrame({CONFIG["COL_USER"]:test_users}), 
                        top_k=CONFIG["SESSION_RECOMMENDATION_SIZE"]//2,
                       remove_seen = True)

,user_id,item_id,prediction
0,bignphat,16739500035,0.004016
1,bignphat,18107316966,0.000769


In [41]:
from collections import deque


In [42]:
q = deque([1,2,3])

In [57]:
IndexError("wcqasd")

IndexError('wcqasd')

In [60]:
try:
    item = q.popleft()
except IndexError:
    raise IndexError("fda")
    

IndexError: fda

In [64]:
chat_id = "123"
by = "popularity"
user_id = None
item_id = None

In [65]:
f"{chat_id}_{by}_{user_id}_{item_id}"

'123_popularity_None_None'